In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Use the citipy module to determine city based on latitude & longitude
from citipy import citipy

In [3]:
# Import the requests library
import requests

# Import the OpenWeather API key
from config import weather_api_key

In [4]:
# Import the datetime module from the datetime library 
# to convert the UTC date to a date format with YYYY-MM-DD-HH-MM-SS
from datetime import datetime

In [5]:
# Import the time module to get today's date
import time

In [6]:
# Import linear regression from the SciPy stats module
from scipy.stats import linregress

In [7]:
# Create a set of random latitude & longitude combinations

# 1. Declare each array for latitude & longitude as a variable
lats = np.random.uniform(low=-90.000, high=90.000, size=200)
lngs = np.random.uniform(low=-90.000, high=90.000, size=200)
# 2. Pack the lats and lngs as pairs by zipping them: 
# there will be 1500 tuples of paired lats and lngs
lat_lngs = zip(lats, lngs)

In [8]:
# Add the latitudes & longitudes to a list
coordinates = list(lat_lngs)

In [9]:
# Iterate through our zipped lat_lngs tuple and find the nearest city

# 1. Create a list for holding the city names
cities = []

# 2. Identify the nearest city for each lat_lng combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # 3. If the city is unique, add it to the cities list
    if city not in cities:
        cities.append(city)

# 4. Print the city count to confirm sufficient count
len(cities)

130

In [10]:
# Create a base URL for the API call & set to a variable 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [34]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our "cities" list
# use the enumerate() method as an alt. way to iterate through list & recieve both index and city
for i, city in enumerate(cities):
    
    # Group the cities in sets of 50 for logging purposes
    # Check if the remainder of the index divided by 50 == 0
    # Check if the index is greater than or equal to 50
    if (i % 50 == 0 and i >=50):
        set_count +=1
        record_count =1
    # Create endpoint URL with each city
    # remove the blank spaces in the city name and concatenate the city name
    city_url = url + "&q=" + city.replace(" ", "+")
    # Log the URL, record count, set count and the city being processed
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count before the next city is processed
    record_count +=1
    
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        city_rain = city_weather["rain"]["1hr"]
        city_snow = city_weather["snow"]["1hr"]
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city_data list with a dictionary for that city
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description" : city_description,
                          "Rain (inches)" : city_rain,
                          "Snow (inches)" : city_snow
                         })
    
    # If a KeyError is experienced, replace value with 0
    except KeyError:
        if city_weather["rain"]["1hr"] != "" or city_weather["snow"]["1hr"] != "":
            city_weather["rain"]["1hr"] = 0
            city_weather["snow"]["1hr"] = 0 
    # If an error is experienced, skip the city
    except Exception:
        print("City not found. Skipping...")
        # pass is a general purpose statement to handle all errors encountered
        # will allow the program to continue
        pass
    
# Indicate the Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cuamba


KeyError: 'rain'

In [32]:
# Tally the number of cities in the city_data array of dictionaries 
# check to make sure you got more than 500 cities
len(city_data)

117

In [33]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Cuamba,-14.80,36.54,65.21,77,0,8.30,MZ,2020-07-12 21:08:46,clear sky
1,Jamestown,42.10,-79.24,75.00,60,1,12.75,US,2020-07-12 21:05:32,clear sky
2,Chuy,-33.70,-53.46,48.54,72,10,8.97,UY,2020-07-12 21:05:48,clear sky
3,Mandera,3.94,41.87,80.76,62,62,19.84,KE,2020-07-12 21:08:46,broken clouds
4,Ushuaia,-54.80,-68.30,35.60,94,40,16.11,AR,2020-07-12 21:08:33,scattered clouds
5,Cayenne,4.93,-52.33,84.20,66,20,9.17,GF,2020-07-12 21:08:47,few clouds
6,Brae,60.40,-1.35,51.80,93,99,5.82,GB,2020-07-12 21:08:47,overcast clouds
7,East London,-33.02,27.91,59.58,55,0,18.79,ZA,2020-07-12 21:08:47,clear sky
8,Solvychegodsk,61.33,46.92,59.88,72,88,2.95,RU,2020-07-12 21:08:47,overcast clouds
9,Qaanaaq,77.48,-69.36,47.61,58,86,3.80,GL,2020-07-12 21:08:42,overcast clouds


In [18]:
# Reorder the columns as:
# City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed

# Assign a variable to an array of the columns in the order we want
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

# Assign the df the new column order
city_data_df = city_data_df[new_column_order]

# Display the DataFrame
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cuamba,MZ,-14.80,36.54,66.15,75,0,10.74,clear sky
1,Jamestown,US,42.10,-79.24,75.99,53,1,17.22,clear sky
2,Chuy,UY,-33.70,-53.46,51.93,68,0,8.52,clear sky
3,Mandera,KE,3.94,41.87,82.85,55,37,18.32,scattered clouds
4,Ushuaia,AR,-54.80,-68.30,35.60,94,40,16.11,scattered clouds
...,...,...,...,...,...,...,...,...,...
112,Ibra,OM,22.69,58.53,84.13,66,58,5.59,broken clouds
113,Knysna,ZA,-34.04,23.05,68.00,24,0,23.04,clear sky
114,Coquimbo,CL,-29.95,-71.34,60.80,72,27,10.29,scattered clouds
115,Blind River,CA,46.18,-82.96,71.60,60,90,9.17,overcast clouds
